In [61]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import plotly.express as px


In [62]:
#Function to convert csv files to dataframes
def process_df(paths: list):
    Dataframes = {}
    for path in paths:
        #os.path.basename(path) gets the file name from the full directory ex. 'dailyActivity_merged.csv' split text separates the name from the '.csv' and [0] takes the first element which is just the name
        name = os.path.splitext(os.path.basename(path))[0] 
        try:
            df = pd.read_csv(path)
            Dataframes[name] = df
        except Exception as e:
            print(f"Failed to process {name}", e)

    return Dataframes
#preliminary clean up
def Quick_df_Check(name: str, df: pd.DataFrame):
    try:
        print(f"---{name}--- \n")
        display(df.head())
        print('-----info-----')
        print(df.info())
        print('-----duplicated-----')
        print(df.duplicated().sum())
        print('-----isnull-----')
        print(df.isnull().sum())
        print('-----isNa-----')
        print(df.isna().sum())
    except Exception as e:
        print(f"Failed to quick check dataframe", e)


In [63]:
#Build custom dataframes functions
def build_Daily_Log(dataframe: pd.DataFrame):
    try:
        src = dataframe
        #ensuring datetime
        src['ActivityDate'] = pd.to_datetime(src['ActivityDate'], errors = 'coerce')
        src['ActivityDate'] = src['ActivityDate'].dt.date
        src['TotalDistance'] = src['TotalDistance'].round(2)
        df = src[['Id','ActivityDate','TotalSteps','TotalDistance','VeryActiveDistance','ModeratelyActiveDistance','LightActiveDistance','VeryActiveMinutes','FairlyActiveMinutes','LightlyActiveMinutes','SedentaryMinutes','Calories']].drop_duplicates()
        #df = df.set_index(['Id', 'ActivityDate'])
    except Exception as e:
        print(f"Failed to build {dataframe}", e)
    
    return df
    

def build_Sleep_Log(dataframe: pd.DataFrame):
    try:
        src = dataframe
        #this was breaking the code because SleepDay is a string/object so dt.normalize raises an error
        src['SleepDay'] = pd.to_datetime(src['SleepDay'],errors='coerce')
        src['Date'] = src['SleepDay'].dt.date
        df = src[['Id','Date','TotalMinutesAsleep','TotalTimeInBed']].drop_duplicates().rename(columns={'TotalMinutesAsleep':'MinutesAsleep','TotalTimeInBed':'TimeInBed'}) 

    except Exception as e:
        print(f"Failed to build {dataframe}", e)
        
    return df

def build_Weight_Log(dataframe: pd.DataFrame):
    try:
        src = dataframe
        src['Date'] = pd.to_datetime(src['Date'], errors ='coerce')
        src['Date'] = src['Date'].dt.date
        src['WeightPounds'] = src['WeightPounds'].round(2)
        src['BMI'] = src['BMI'].round(2)
        df = src[['Id','Date','WeightPounds','BMI']].drop_duplicates()

    except Exception as e:
        print(f"Failed to build {dataframe}", e)
    
    return df
    

def build_Heartrate_Log(dataframe: pd.DataFrame):
    try:
        src = dataframe
        src['Time'] = pd.to_datetime(src['Time'], errors ='coerce')
        src['Date'] = src['Time'].dt.date
        df = src[['Id','Date','Time','Value']].drop_duplicates().rename(columns={'Value':'Heartrate'})

    except Exception as e:
        print(f"Failed to build {dataframe}", e)
    
    return df

def build_Hourly_Intensity_Log(dataframe: pd.DataFrame):
    try:
        src = dataframe
        src['ActivityHour'] = pd.to_datetime(src['ActivityHour'], errors = 'coerce')
        src['Date'] = src['ActivityHour'].dt.date
        #excluding rows with zero intensity src = src[mask], boolean mask: src['TotalIntensity'] != 0
        src = src[src['TotalIntensity'] != 0]
        df = src[['Id','Date','ActivityHour','TotalIntensity']].drop_duplicates().rename(columns={'TotalIntensity':'Intensity'})

    except Exception as e:
        print(f"Failed to build {dataframe}", e)

    return df

In [64]:
#IQR outlier filter
def iqr_bounds(s:pd.DataFrame):
    try:
        #x axis bounds
        q1, q3 = s.quantile([0.25, 0.75])
        IQR = q3 - q1
        return q1 -1.5*IQR , q3 + 1.5*IQR
    
    except Exception as e:
        print(f"failed irq bounds", e)

def irq_filter(df:pd.DataFrame,x:pd.DataFrame,y:pd.DataFrame):
    try:
        x_low, x_high = iqr_bounds(x)
        y_low, y_high = iqr_bounds(y)
#like list comprehension
        filtered = df[(x >= x_low) & (x<= x_high) & (y >= y_low) & (y <= y_high)]
        return filtered
    except Exception as e:
        print(f"failed iqr filter", e)

In [65]:
#getting path to data folder
notebooks_dir = os.getcwd() #cwd = current working directory ../Fitbit_Data_Analysis/Notebooks
data_dir = os.path.abspath(os.path.join(notebooks_dir, '..','Data','Fitbase_Data')) #getting path to the data

#checks
if not os.path.isdir(data_dir):
    raise FileNotFoundError(f"Directory not found: {data_dir}")

#addihng all CSVs to a list
csv_paths = sorted(glob.glob(os.path.join(data_dir, '*.csv')))

#check
if not csv_paths:
    raise FileNotFoundError(f"No CSV files found in {data_dir}")

#double checking type
print(type(csv_paths))

<class 'list'>


In [66]:
#pass in csv_paths list into process_df function that creates the dataframes
Dataframes = process_df(csv_paths)
#checking everything worked 
for name, df in Dataframes.items():
    print("-----------------------------------------------------------")
    print(f"{name}")
    print("-----------------------------------------------------------")
    print(df.info())


-----------------------------------------------------------
dailyActivity_merged
-----------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        940 non-null    int64  
 1   ActivityDate              940 non-null    object 
 2   TotalSteps                940 non-null    int64  
 3   TotalDistance             940 non-null    float64
 4   TrackerDistance           940 non-null    float64
 5   LoggedActivitiesDistance  940 non-null    float64
 6   VeryActiveDistance        940 non-null    float64
 7   ModeratelyActiveDistance  940 non-null    float64
 8   LightActiveDistance       940 non-null    float64
 9   SedentaryActiveDistance   940 non-null    float64
 10  VeryActiveMinutes         940 non-null    int64  
 11  FairlyActiveMinutes       940 non-

In [67]:
#checking the value range of intensity
display(sorted(Dataframes['hourlyIntensities_merged'].TotalIntensity.unique()))
#viewing dataframe values
for name, df in Dataframes.items():
    print('---------------------------------------------------')
    print(f"{name} ({df.shape[0]} rows, {df.shape[1]} cols)")
    print('---------------------------------------------------')
    display(df.head(100))



[np.int64(0),
 np.int64(1),
 np.int64(2),
 np.int64(3),
 np.int64(4),
 np.int64(5),
 np.int64(6),
 np.int64(7),
 np.int64(8),
 np.int64(9),
 np.int64(10),
 np.int64(11),
 np.int64(12),
 np.int64(13),
 np.int64(14),
 np.int64(15),
 np.int64(16),
 np.int64(17),
 np.int64(18),
 np.int64(19),
 np.int64(20),
 np.int64(21),
 np.int64(22),
 np.int64(23),
 np.int64(24),
 np.int64(25),
 np.int64(26),
 np.int64(27),
 np.int64(28),
 np.int64(29),
 np.int64(30),
 np.int64(31),
 np.int64(32),
 np.int64(33),
 np.int64(34),
 np.int64(35),
 np.int64(36),
 np.int64(37),
 np.int64(38),
 np.int64(39),
 np.int64(40),
 np.int64(41),
 np.int64(42),
 np.int64(43),
 np.int64(44),
 np.int64(45),
 np.int64(46),
 np.int64(47),
 np.int64(48),
 np.int64(49),
 np.int64(50),
 np.int64(51),
 np.int64(52),
 np.int64(53),
 np.int64(54),
 np.int64(55),
 np.int64(56),
 np.int64(57),
 np.int64(58),
 np.int64(59),
 np.int64(60),
 np.int64(61),
 np.int64(62),
 np.int64(63),
 np.int64(64),
 np.int64(65),
 np.int64(66),
 np.i

---------------------------------------------------
dailyActivity_merged (940 rows, 15 cols)
---------------------------------------------------


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1844505072,4/15/2016,3844,2.54,2.54,0.0,0.00,0.00,2.54,0.0,0,0,176,527,1725
96,1844505072,4/16/2016,3414,2.26,2.26,0.0,0.00,0.00,2.26,0.0,0,0,147,1293,1657
97,1844505072,4/17/2016,4525,2.99,2.99,0.0,0.14,0.26,2.59,0.0,2,8,199,1231,1793
98,1844505072,4/18/2016,4597,3.04,3.04,0.0,0.00,0.48,2.56,0.0,0,12,217,1211,1814


---------------------------------------------------
dailyCalories_merged (940 rows, 3 cols)
---------------------------------------------------


,Id,ActivityDay,Calories
0,1503960366,4/12/2016,1985
1,1503960366,4/13/2016,1797
2,1503960366,4/14/2016,1776
3,1503960366,4/15/2016,1745
4,1503960366,4/16/2016,1863
...,...,...,...
95,1844505072,4/15/2016,1725
96,1844505072,4/16/2016,1657
97,1844505072,4/17/2016,1793
98,1844505072,4/18/2016,1814


...

,Id,ActivityHour,Steps00,Steps01,Steps02,Steps03,Steps04,Steps05,Steps06,Steps07,...,Steps50,Steps51,Steps52,Steps53,Steps54,Steps55,Steps56,Steps57,Steps58,Steps59
0,1503960366,4/13/2016 12:00:00 AM,4,16,0,0,0,9,0,17,...,0,9,8,0,20,1,0,0,0,0
1,1503960366,4/13/2016 1:00:00 AM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1503960366,4/13/2016 2:00:00 AM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1503960366,4/13/2016 3:00:00 AM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1503960366,4/13/2016 4:00:00 AM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1503960366,4/16/2016 11:00:00 PM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,1503960366,4/17/2016 12:00:00 AM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,1503960366,4/17/2016 1:00:00 AM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,1503960366,4/17/2016 2:00:00 AM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---------------------------------------------------
sleepDay_merged (413 rows, 5 cols)
---------------------------------------------------


,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016 12:00:00 AM,1,327,346
1,1503960366,4/13/2016 12:00:00 AM,2,384,407
2,1503960366,4/15/2016 12:00:00 AM,1,412,442
3,1503960366,4/16/2016 12:00:00 AM,2,340,367
4,1503960366,4/17/2016 12:00:00 AM,1,700,712
...,...,...,...,...,...
95,3977333714,4/26/2016 12:00:00 AM,1,250,371
96,3977333714,4/27/2016 12:00:00 AM,1,349,540
97,3977333714,4/28/2016 12:00:00 AM,1,261,423
98,3977333714,4/29/2016 12:00:00 AM,1,333,478


---------------------------------------------------
weightLogInfo_merged (67 rows, 8 cols)
---------------------------------------------------


,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,5/2/2016 11:59:59 PM,52.599998,115.963147,22.0,22.650000,True,1462233599000
1,1503960366,5/3/2016 11:59:59 PM,52.599998,115.963147,NaN,22.650000,True,1462319999000
2,1927972279,4/13/2016 1:08:52 AM,133.500000,294.317120,NaN,47.540001,False,1460509732000
3,2873212765,4/21/2016 11:59:59 PM,56.700001,125.002104,NaN,21.450001,True,1461283199000
4,2873212765,5/12/2016 11:59:59 PM,57.299999,126.324875,NaN,21.690001,True,1463097599000
...,...,...,...,...,...,...,...,...
62,8877689391,5/6/2016 6:43:35 AM,85.000000,187.392923,NaN,25.440001,False,1462517015000
63,8877689391,5/8/2016 7:35:53 AM,85.400002,188.274775,NaN,25.559999,False,1462692953000
64,8877689391,5/9/2016 6:39:44 AM,85.500000,188.495234,NaN,25.610001,False,1462775984000
65,8877689391,5/11/2016 6:51:47 AM,85.400002,188.274775,NaN,25.559999,False,1462949507000


BUILDING RELATIONAL TABLES

In [68]:
conn = sqlite3.connect('../Data/fitbitdata.db')

Daily_Log_df = build_Daily_Log(Dataframes['dailyActivity_merged'].copy())
Quick_df_Check('Activity Log',Daily_Log_df)

Sleep_Log_df = build_Sleep_Log(Dataframes['sleepDay_merged'].copy())
Quick_df_Check('Sleep Log',Sleep_Log_df)

Weight_Log_df = build_Weight_Log(Dataframes['weightLogInfo_merged'].copy())
Quick_df_Check('Weight Log', Weight_Log_df)

Heartrate_Log_df = build_Heartrate_Log(Dataframes['heartrate_seconds_merged'].copy())
Quick_df_Check('Heartrate Log',Heartrate_Log_df)

Hourly_Intensity_df = build_Hourly_Intensity_Log(Dataframes['hourlyIntensities_merged'].copy())
Quick_df_Check('Hourly Intensity',Hourly_Intensity_df)

Daily_Log_df.to_sql('Daily_Activity', conn, index = False, if_exists='replace')
Sleep_Log_df.to_sql('Sleep_Log', conn, index = False, if_exists='replace')
Weight_Log_df.to_sql('Weight_Log', conn, index = False, if_exists='replace')
Heartrate_Log_df.to_sql('Heartrate_Log', conn, index = False, if_exists='replace')
Hourly_Intensity_df.to_sql('Hourly_Intensity', conn, index = False, if_exists='replace')
conn.commit()

---Activity Log--- 



,Id,ActivityDate,TotalSteps,TotalDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,2016-04-12,13162,8.50,1.88,0.55,6.06,25,13,328,728,1985
1,1503960366,2016-04-13,10735,6.97,1.57,0.69,4.71,21,19,217,776,1797
2,1503960366,2016-04-14,10460,6.74,2.44,0.40,3.91,30,11,181,1218,1776
3,1503960366,2016-04-15,9762,6.28,2.14,1.26,2.83,29,34,209,726,1745
4,1503960366,2016-04-16,12669,8.16,2.71,0.41,5.04,36,10,221,773,1863


-----info-----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        940 non-null    int64  
 1   ActivityDate              940 non-null    object 
 2   TotalSteps                940 non-null    int64  
 3   TotalDistance             940 non-null    float64
 4   VeryActiveDistance        940 non-null    float64
 5   ModeratelyActiveDistance  940 non-null    float64
 6   LightActiveDistance       940 non-null    float64
 7   VeryActiveMinutes         940 non-null    int64  
 8   FairlyActiveMinutes       940 non-null    int64  
 9   LightlyActiveMinutes      940 non-null    int64  
 10  SedentaryMinutes          940 non-null    int64  
 11  Calories                  940 non-null    int64  
dtypes: float64(4), int64(7), object(1)
memory usage: 88.3+ KB
None
-----duplicated-----
0
-----isnull-----
Id

C:\Users\migue\AppData\Local\Temp\ipykernel_13916\2043128897.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,Id,Date,MinutesAsleep,TimeInBed
0,1503960366,2016-04-12,327,346
1,1503960366,2016-04-13,384,407
2,1503960366,2016-04-15,412,442
3,1503960366,2016-04-16,340,367
4,1503960366,2016-04-17,700,712


...

-----info-----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483658 entries, 0 to 2483657
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Id         int64         
 1   Date       object        
 2   Time       datetime64[ns]
 3   Heartrate  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 75.8+ MB
None
-----duplicated-----
0
-----isnull-----
Id           0
Date         0
Time         0
Heartrate    0
dtype: int64
-----isNa-----
Id           0
Date         0
Time         0
Heartrate    0
dtype: int64


C:\Users\migue\AppData\Local\Temp\ipykernel_13916\2043128897.py:60: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



---Hourly Intensity--- 



,Id,Date,ActivityHour,Intensity
0,1503960366,2016-04-12,2016-04-12 00:00:00,20
1,1503960366,2016-04-12,2016-04-12 01:00:00,8
2,1503960366,2016-04-12,2016-04-12 02:00:00,7
8,1503960366,2016-04-12,2016-04-12 08:00:00,13
9,1503960366,2016-04-12,2016-04-12 09:00:00,30


-----info-----
<class 'pandas.core.frame.DataFrame'>
Index: 13002 entries, 0 to 22098
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Id            13002 non-null  int64         
 1   Date          13002 non-null  object        
 2   ActivityHour  13002 non-null  datetime64[ns]
 3   Intensity     13002 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 507.9+ KB
None
-----duplicated-----
0
-----isnull-----
Id              0
Date            0
ActivityHour    0
Intensity       0
dtype: int64
-----isNa-----
Id              0
Date            0
ActivityHour    0
Intensity       0
dtype: int64


In [69]:
#activity vs sleep
#multiplying by 1.0 forces floating-point division, without it SQL might truncate decimals
query1 = """
    SELECT a.Id, a.TotalSteps, a.Calories, s.MinutesAsleep, s.TimeInBed,
    CASE WHEN s.TimeInBed > 0
        THEN (1.0 * s.MinutesAsleep / s.TimeInBed) * 100
        ELSE NULL
    END AS SleepEfficiency
    FROM Daily_Activity a
    JOIN Sleep_Log s
    ON a.Id = s.Id AND a.ActivityDate = s.Date;
    """
result1 = pd.read_sql(query1, conn)
result1

,Id,TotalSteps,Calories,MinutesAsleep,TimeInBed,SleepEfficiency
0,1503960366,13162,1985,327,346,94.508671
1,1503960366,10735,1797,384,407,94.348894
2,1503960366,9762,1745,412,442,93.212670
3,1503960366,12669,1863,340,367,92.643052
4,1503960366,9705,1728,700,712,98.314607
...,...,...,...,...,...,...
405,8792009665,7174,2896,343,360,95.277778
406,8792009665,1619,1962,503,527,95.445920
407,8792009665,1831,2015,415,423,98.108747
408,8792009665,2421,2297,516,545,94.678899


In [70]:
fig1 = px.scatter(
    result1, x='Calories', y='SleepEfficiency',
    hover_data=['Id','Calories','SleepEfficiency','MinutesAsleep','TimeInBed'],
    trendline='ols', opacity=0.8
)
fig1.update_layout(title='Calories Spent vs Sleep Efficiency')
fig1.show()


In [71]:
#manually removing outliers
exclude_ids = [3977333714,1844505072]
result1_excluded = result1[~result1['Id'].isin(exclude_ids)]
fig1_excluded = px.scatter(
    result1_excluded, x='Calories', y='SleepEfficiency',
    hover_data=['Id','Calories','SleepEfficiency','MinutesAsleep','TimeInBed'],
    trendline='ols', opacity=0.8
)
fig1_excluded.update_layout(title='Calories Spent vs Sleep Efficiency')
fig1_excluded.show()

In [72]:
#using iqr_filter
result1_filtered = irq_filter(result1,result1['Calories'],result1['SleepEfficiency'])
fig2 = px.scatter(
    result1_filtered, x='Calories', y='SleepEfficiency',
    hover_data=['Id','Calories','SleepEfficiency','MinutesAsleep','TimeInBed'],
    trendline='ols', opacity=0.8
)
fig2.update_layout(title='Calories Spent vs Sleep Efficiency')
fig2.show()


What type of workout intensity burns more calories

In [73]:
display(Hourly_Intensity_df)

,Id,Date,ActivityHour,Intensity
0,1503960366,2016-04-12,2016-04-12 00:00:00,20
1,1503960366,2016-04-12,2016-04-12 01:00:00,8
2,1503960366,2016-04-12,2016-04-12 02:00:00,7
8,1503960366,2016-04-12,2016-04-12 08:00:00,13
9,1503960366,2016-04-12,2016-04-12 09:00:00,30
...,...,...,...,...
22094,8877689391,2016-05-12,2016-05-12 10:00:00,12
22095,8877689391,2016-05-12,2016-05-12 11:00:00,29
22096,8877689391,2016-05-12,2016-05-12 12:00:00,93
22097,8877689391,2016-05-12,2016-05-12 13:00:00,6


In [74]:
#todo:
#handle outliers, wrong data might be affecting results
query2 = """
    WITH avg_DailyIntensity AS (
    SELECT Id,Date, AVG(Intensity) AS AverageIntensity
    FROM hourly_Intensity
    WHERE Intensity != 0
    GROUP BY Id, Date
    )
    SELECT 
        da.Id, da.ActivityDate, da.Calories, da.TotalSteps,
        COALESCE(VeryActiveMinutes, 0) + COALESCE(FairlyActiveMinutes, 0) + COALESCE(LightlyActiveMinutes, 0) AS ActiveMinutes,
        adi.AverageIntensity
    FROM Daily_Activity da
    JOIN avg_DailyIntensity adi ON da.Id = adi.Id AND da.ActivityDate = adi.Date
    ORDER BY da.Calories;
    """
result2 = pd.read_sql(query2, conn)

display(result2[['Id','ActivityDate','TotalSteps','Calories','AverageIntensity','ActiveMinutes']])


,Id,ActivityDate,TotalSteps,Calories,AverageIntensity,ActiveMinutes
0,4319703577,2016-05-12,17,257,1.000000,2
1,2347167796,2016-04-29,42,403,1.000000,4
2,5553957443,2016-05-12,3121,741,14.000000,62
3,6962181067,2016-05-12,3587,928,24.200000,113
4,1624580081,2016-05-12,2971,1002,9.900000,107
...,...,...,...,...,...,...
851,5577150313,2016-04-30,12363,4501,46.000000,415
852,5577150313,2016-05-01,13368,4546,47.578947,444
853,8877689391,2016-04-16,29326,4547,45.235294,552
854,5577150313,2016-04-17,12231,4552,39.666667,396


In [75]:
fig = px.scatter(
    result2, x='ActiveMinutes', y='Calories',
    color='AverageIntensity',
    hover_data=['Id','ActivityDate','ActiveMinutes','Calories','AverageIntensity'],
    trendline='ols', opacity=0.7
)
fig.update_layout(title='Calories vs ActiveMinutes')
fig.show()

In [88]:
r2 = result2.dropna(subset=['AverageIntensity','TotalSteps','ActiveMinutes','Calories']).copy()

r2['IntensityQuintile'] = pd.qcut(r2['AverageIntensity'], 5, labels=['Q1','Q2','Q3','Q4','Q5'])
#df with  mean calories vs average intensity, grouped by intensity group
df_q = (r2.groupby('IntensityQuintile', observed=True)['Calories']
          .mean().reset_index())

fig_q = px.bar(df_q, x='IntensityQuintile', y='Calories',
               title='Mean Calories by Average Intensity Quintile (Q1=lowest)')
fig_q.show()

r2['MinutesBin'] = pd.qcut(r2['ActiveMinutes'], 4, labels=['Low','Med-Low','Med-High','High'])
df = (r2.groupby(['MinutesBin','IntensityQuintile'], observed=True)['Calories']
              .mean().reset_index())
fig = px.bar(df, x='IntensityQuintile', y='Calories',
                   facet_col='MinutesBin', facet_col_wrap=2,
                   category_orders={'IntensityQuintile':['Q1','Q2','Q3','Q4','Q5']},
                   title='Mean Calories by Intensity, stratified by ActiveMinutes')
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))
fig.show()

10k steps low vs high intensity


In [85]:
#aggregating to mean calories at each step count per intensity group
df_agg = (r2.groupby(['IntensityQuintile','TotalSteps'], observed=True)['Calories']
            .mean().reset_index()
            .sort_values(['IntensityQuintile','TotalSteps'])
)
fig = px.line(
        df_agg, x='TotalSteps', y='Calories',
        color='IntensityQuintile', markers=True,
        title='Calories vs Steps by Intensity group'
    )
fig.show()

In [87]:
#trying running iqr to remove outliers
r2_iqr = irq_filter(r2, r2['TotalSteps'], r2['Calories'])

df_agg = (r2_iqr.groupby(['IntensityQuintile','TotalSteps'], observed=True)['Calories']
            .mean().reset_index()
            .sort_values(['IntensityQuintile','TotalSteps']))

fig = px.line(
    df_agg, x='TotalSteps', y='Calories',
    color='IntensityQuintile', markers=True,
    title='Calories vs Steps by Intensity (IQR-filtered)'
)
fig.show()

In [94]:
fig = px.scatter(
        r2, x='TotalSteps', y='Calories',
        color='IntensityQuintile',opacity=0.8, trendline='lowess',
        title='Calories vs Steps by Intensity group'
    )
fig.show()

Making a correlation visualization to see which field affects weight change the most

In [96]:
query3 = """
    WITH avg_DailyIntensity AS (
    SELECT Id,Date, AVG(Intensity) AS AverageIntensity
    FROM hourly_Intensity
    WHERE Intensity != 0
    GROUP BY Id, Date
    ),
    Sleep_efficiency AS (
    SELECT Id, Date,
    CASE WHEN TimeInBed > 0
        THEN (1.0 * MinutesAsleep / TimeInBed) * 100
        ELSE NULL
    END AS SleepEfficiency
    FROM Sleep_Log
    )
    SELECT 
        da.Id, da.ActivityDate, da.Calories, da.TotalSteps,
        COALESCE(VeryActiveMinutes, 0) + COALESCE(FairlyActiveMinutes, 0) + COALESCE(LightlyActiveMinutes, 0) AS ActiveMinutes,
        adi.AverageIntensity,
        s.SleepEfficiency,
        w.WeightPounds
    FROM Daily_Activity da
    JOIN avg_DailyIntensity adi ON da.Id = adi.Id AND da.ActivityDate = adi.Date
    LEFT JOIN Sleep_efficiency s ON s.Id = da.Id AND s.Date = da.ActivityDate
    LEFT JOIN Weight_Log w ON w.Id = da.Id AND w.Date = da.ActivityDate
    ORDER BY da.Calories;
    """
result3 = pd.read_sql(query3, conn)
display(result3)

,Id,ActivityDate,Calories,TotalSteps,ActiveMinutes,AverageIntensity,SleepEfficiency,WeightPounds
0,4319703577,2016-05-12,257,17,2,1.000000,94.080997,NaN
1,2347167796,2016-04-29,403,42,4,1.000000,86.892178,NaN
2,5553957443,2016-05-12,741,3121,62,14.000000,92.210526,NaN
3,6962181067,2016-05-12,928,3587,113,24.200000,96.448598,136.47
4,1624580081,2016-05-12,1002,2971,107,9.900000,NaN,NaN
...,...,...,...,...,...,...,...,...
851,5577150313,2016-04-30,4501,12363,415,46.000000,95.150115,NaN
852,5577150313,2016-05-01,4546,13368,444,47.578947,95.226131,NaN
853,8877689391,2016-04-16,4547,29326,552,45.235294,NaN,188.50
854,5577150313,2016-04-17,4552,12231,396,39.666667,94.168096,199.96
